<a href="https://colab.research.google.com/github/ishancoderr/Neural-networks/blob/main/Boltzman_machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
!pip install -q torch==1.0.0 torchvision


In [78]:
import torch
print(torch.__version__)

1.0.0


In [79]:
!pip install torch

In [81]:
import numpy as np
import pandas as pd


In [82]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable


In [83]:
url="https://raw.githubusercontent.com/ishancoderr/Neural-networks/main/movies.dat"
url2="https://raw.githubusercontent.com/ishancoderr/Neural-networks/main/users.dat"
url3="https://raw.githubusercontent.com/ishancoderr/Neural-networks/main/ratings.dat"
movies=pd.read_csv(url,sep='::',header=None,engine='python',encoding='latin-1')
users=pd.read_csv(url2,sep='::',header=None,engine='python',encoding='latin-1')
ratings=pd.read_csv(url3,sep='::',header=None,engine='python',encoding='latin-1')

In [84]:
url4="https://raw.githubusercontent.com/ishancoderr/Neural-networks/main/u1.base"
trining_set =pd.read_csv(url4,delimiter='\t')
trining_set=np.array(trining_set,dtype='int')
test_set=pd.read_csv(url4,delimiter='\t')
test_set=np.array(test_set,dtype='int')

**Getting the number of users and movies**

In [85]:
nb_users=int(max(max(trining_set[:,0]),max(test_set[:,0])))
nb_movies=int(max(max(trining_set[:,1]),max(test_set[:,1])))

In [86]:
def convert(data):
  new_data =[]
  for id_users in range(1,nb_users + 1):
    id_movies=data[:,1][data[:,0]==id_users]
    id_ratings=data[:,1][data[:,0]==id_users]
    ratings=np.zeros(nb_movies)
    ratings[id_movies-1]=id_ratings
    new_data.append(ratings)
  return new_data
trining_set=convert(trining_set)
test_set=convert(test_set)  



In [87]:
trining_set = torch.FloatTensor(trining_set)
test_set = torch.FloatTensor(test_set)

In [88]:
trining_set[trining_set==0]=-1
trining_set[trining_set==1]=0
trining_set[trining_set==2]=0
trining_set[trining_set>=3]=1
test_set[test_set==0]=-1
test_set[test_set==1]=0
test_set[test_set==2]=0
test_set[test_set>=3]=1

In [92]:
class RBM():
  def __init__(self,nv,nh):
    self.W = torch.randn(nh,nv)
    self.a = torch.random(1,nh)
    self.b = torch.random(1,nv)
  def sample_h(self, x):
    wx = torch.mm(x,self.W.t())
    activation =wx+self.a.expand_as(wx)
    p_h_given_v =torch.sigmoid(activation)
    return p_h_given_v,torch.bernoulli(p_h_given_v)
  def sample_v(self,x):
    wx = torch.mm(x,self.W.t())
    activation= wx+ self.a.expand_as(wx)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h,torch.bernoulli(p_v_given_h)
  def train(self,v0,vk,ph0,phk):
    self.w+=torch.mm(v0.t(),ph0)-torch.mm(vk.t(),phk)
    self.b+